# Pandas 迭代

In [1]:
# 03_learn_pandas_iteration
#
# created by LuYF-Lemon-love <luyanfeng_nlp@qq.com> on March 2, 2022
# updated by LuYF-Lemon-love <luyanfeng_nlp@qq.com> on March 2, 2023
#
# 参考文档链接: https://www.pypandas.cn/docs/getting_started/basics.html#%E8%BF%AD%E4%BB%A3

## 导入 Pandas 与 NumPy

In [2]:
import numpy as np
import pandas as pd

Pandas 对象基于类型进行迭代操作。**Series** 迭代时被视为数组，基础迭代生成值。**DataFrame** 则遵循字典式习语，用对象的 key 实现迭代操作。

简言之，基础迭代（for i in object）生成：

- Series ：值

- DataFrame：列标签

例如，DataFrame 迭代时输出列名：

In [3]:
df = pd.DataFrame({'col1': np.random.randn(3), 'col2': np.random.randn(3)}, index=['a', 'b', 'c'])
df

,col1,col2
a,0.263700,0.407105
b,0.037859,1.096099
c,0.639418,-1.324818


In [4]:
for col in df:
    print(col)

col1
col2


Pandas 对象还支持字典式的 **items()** 方法，通过键值对迭代。

用下列方法可以迭代 DataFrame 里的行：

- **iterrows()**：把 DataFrame 里的行当作 （index， Series）对进行迭代。该操作把行转为 Series，同时改变数据类型，并对性能有影响。

- **itertuples()**: 把 DataFrame 的行当作值的命名元组进行迭代。该操作比 iterrows() 快的多，建议尽量用这种方法迭代 DataFrame 的值。

Pandas 对象迭代的速度较慢。

**警告:** 永远不要修改迭代的内容，这种方式不能确保所有操作都能正常运作。基于数据类型，迭代器返回的是复制（copy）的结果，不是视图（view），这种写入可能不会生效！

下例中的赋值就不会生效：

In [5]:
df = pd.DataFrame({'a': [1, 2, 3], 'b': ['a', 'b', 'c']})
df

,a,b
0,1,a
1,2,b
2,3,c


In [6]:
for index, row in df.iterrows():
    row['a'] = 10

df

,a,b
0,1,a
1,2,b
2,3,c


## 项目（items）

与字典型接口类似，**items()** 通过键值对进行迭代：

- **Series**：（Index，标量值）对

- **DataFrame**：（列，Series）对

示例如下：

In [7]:
df

,a,b
0,1,a
1,2,b
2,3,c


In [8]:
for label, ser in df.items():
    print(label)
    print("*" * 66)
    print(ser)
    print("\n\n")

a
******************************************************************
0    1
1    2
2    3
Name: a, dtype: int64



b
******************************************************************
0    a
1    b
2    c
Name: b, dtype: object





## iterrows

**iterrows():** 迭代 DataFrame 或 Series 里的每一行数据。这个操作返回一个迭代器，生成索引值及包含每行数据的 **Series** 。

In [9]:
df

,a,b
0,1,a
1,2,b
2,3,c


In [10]:
for row_index, row in df.iterrows():
    print(row_index, row, sep='\n' + '*' * 66 + '\n')
    print("\n\n")

0
******************************************************************
a    1
b    a
Name: 0, dtype: object



1
******************************************************************
a    2
b    b
Name: 1, dtype: object



2
******************************************************************
a    3
b    c
Name: 2, dtype: object





**注意**

**iterrows()** 返回的是 **Series** 里的每一行数据，**该操作不保留每行数据的数据类型**，因为数据类型是通过 DataFrame 的列界定的。

示例如下：

In [11]:
df_orig = pd.DataFrame([[1, 1.5]], columns=['int', 'float'])
df_orig

,int,float
0,1,1.5


In [12]:
df_orig.dtypes

int        int64
float    float64
dtype: object

In [13]:
row = next(df_orig.iterrows())[1]
row

int      1.0
float    1.5
Name: 0, dtype: float64

row 里的值以 Series 形式返回，并被转换为**浮点数**，原始的整数值则在列 X：

In [14]:
row['int'].dtype

dtype('float64')

In [15]:
df_orig['int'].dtype

dtype('int64')

要想在行迭代时保存数据类型，最好用 **itertuples()**，这个函数返回值的命名元组，总的来说，该操作比 **iterrows()** 速度更快。

下例展示了怎样转置 DataFrame：

In [16]:
df2 = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})
print(df2)

   x  y
0  1  4
1  2  5
2  3  6


In [17]:
print(df2.T)

   0  1  2
x  1  2  3
y  4  5  6


In [18]:
df2_t = pd.DataFrame({idx: values for idx, values in df2.iterrows()})
print(df2_t)

   0  1  2
x  1  2  3
y  4  5  6


## itertuples

**itertuples()** 方法返回为 DataFrame 里每行数据生成命名元组的迭代器。该元组的第一个元素是行的**索引值**，其余的值则是行的值。

示例如下：

In [19]:
df

,a,b
0,1,a
1,2,b
2,3,c


In [20]:
for row in df.itertuples():
    print(row)

Pandas(Index=0, a=1, b='a')
Pandas(Index=1, a=2, b='b')
Pandas(Index=2, a=3, b='c')


该方法不会把行转换为 Series，只是返回命名元组里的值。itertuples() 保存值的数据类型，而且比 iterrows() 快。

**注意**

包含无效 Python 识别符的列名、重复的列名及以下划线开头的列名，会被重命名为位置名称。如果列数较大，比如大于 255 列，则返回正则元组。